In [1]:
import numpy as np

import tensorflow as tf

from nltk.tokenize import word_tokenize

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow import keras

In [2]:
one_step_reloaded = tf.saved_model.load('one_step_batch_second_validation_1')

2022-03-13 16:17:23.387129: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-13 16:17:23.387527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


2022-03-13 16:17:25.524409: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-13 16:17:26.196687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:17:26.202543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
# print(filtered_data[:200])

In [10]:
vocab = sorted(set(word_tokenize(filtered_data)))
print(len(vocab))

10877


In [11]:
ids_from_words = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
words_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

In [12]:
test_data = open('test_events.txt', 'rb').read().decode(encoding='utf-8')

corpus = test_data.lower().split("\n")

story_text = []

events_elapsed = []
events_actual_next = []


for line in corpus:
    if line == 'story start':
        story_text = []
        continue
        
    if line == 'story end':
        actual = list(filter(None, story_text[-1].split(' ')))
        events_actual_next.append(actual)
    
        events_elapsed.append([list(filter(None, ''.join(story_text[:-1]).split(' ')))])
        
        # print(words_from_ids(elapsed_tensor))
        # print(actual)
        
        story_text = []
        continue
    
    story_text.append(line)

In [13]:
print(len(events_elapsed))

1054


In [14]:
skip_ids = ids_from_words(['[UNK]'])[:, None]
sparse_mask = tf.SparseTensor(
    # Put a -inf at each bad index.
    values=[-float('inf')]*len(skip_ids),
    indices=skip_ids,
    # Match the shape to the vocabulary
    dense_shape=[len(ids_from_words.get_vocabulary())])
prediction_mask = tf.sparse.to_dense(sparse_mask)

In [15]:
predicted_events = []

for event_elapsed in events_elapsed:
    
    states = None

    seed_words = [[]]
    for event in event_elapsed[0]:
        seed_words[0].append(event)

    event_predicted_next = []

    for n in range(4):
        seed_w = ids_from_words(seed_words)
        
        inputs = tf.convert_to_tensor(pad_sequences(seed_w, maxlen=19))
        
        predicted_logits, states = one_step_reloaded.generate_one_step(inputs, states=states)
        
        predicted_logits = predicted_logits + prediction_mask
        
        predicted_word = words_from_ids(tf.math.argmax(tf.nn.softmax(predicted_logits[0]), 0))
        
        predicted_word = predicted_word.numpy().decode()

        event_predicted_next.append(predicted_word)

        seed_words[0].append(predicted_word)

        seed_words[0] = seed_words[0][-19:]
    
    predicted_events.append(event_predicted_next)

2022-03-13 16:18:34.922426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:18:35.047515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:18:35.927647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:18:35.960433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [17]:
print(predicted_events)

[['she', 'na', 'was', 'na'], ['he', 'na', 'is', 'happy'], ['he', 'na', 'knew', 'able'], ['tom', 'na', 'knew', 'able'], ['it', 'na', 'felt', 'great'], ['na', 'na', 'is', 'happy'], ['she', 'na', 'is', 'happy'], ['he', 'na', 'is', 'na'], ['she', 'na', 'went', 'na'], ['i', 'na', 'felt', 'sick'], ['they', 'na', 'were', 'na'], ['he', 'na', 'was', 'able'], ['he', 'them', 'avoided', 'na'], ['he', 'eggs', 'loved', 'na'], ['john', 'na', 'decided', 'na'], ['it', 'na', 'is', 'amazing'], ['tom', 'na', 'knew', 'na'], ['i', 'na', 'had', 'na'], ['hannah', 'na', 'is', 'glad'], ['i', 'na', "'ve", 'na'], ['she', 'na', 'was', 'excited'], ['they', 'na', 'were', 'na'], ['she', 'job', 'left', 'him'], ['it', 'her', 'disappointed', 'na'], ['he', 'na', 'did', 'na'], ['na', 'na', 'will', 'na'], ['she', 'it', 'was', 'able'], ['he', 'them', 'avoided', 'na'], ['na', 'na', 'stuck', 'morning'], ['she', 'na', 'decided', 'na'], ['it', 'na', 'is', 'glad'], ['he', 'na', 'returned', 'na'], ['he', 'na', 'is', 'glad'], ['po

In [18]:
#check the model accuracy of model 1 here 

In [19]:
text_translator = tf.saved_model.load('text_translator_2')

2022-03-13 16:20:38.595441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:38.599472: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:38.826333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:38.831853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:38.836029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:38.839261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:38.843653: I tensorflow/core/grappler/optimizers/cust

In [20]:
input_text =[]
# print(events_actual_next)

for el in predicted_events:
    temp = " ".join(el)
    input_text.append(temp)

# print(input_text)
    
input_text = tf.constant(input_text)
print(input_text)


tf.Tensor(
[b'she na was na' b'he na is happy' b'he na knew able' ...
 b'she na was disappointed' b'john na agreed him' b'he them proved na'], shape=(1054,), dtype=string)


In [21]:
# input_text = tf.constant([
#     'She show attended grandparents',
#     'boy crust did NA', 
# ])

# result = text_translator.translate(
#     input_text = input_text)


In [22]:
result = text_translator.tf_translate(
    input_text = input_text)

2022-03-13 16:20:48.810837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:50.978550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:51.227787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:51.713338: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:51.791006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:51.869299: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-13 16:20:51.946368: I tensorflow/core/grappler/optimizers/cust

In [23]:
actual_sentences = []
with open('test_sentences.txt') as ts:
    prev_row = None
    for idx, row in enumerate(ts.readlines()):
        if row == "Story End\n":
            actual_sentences.append(prev_row)
        prev_row = row

In [24]:
print(len(actual_sentences))

1500


In [25]:
# for i in range(len(input_text)):
for i in range(3):
    actual_sentence = actual_sentences[i]
    predicted_sentence = result['text'][i].numpy().decode()
    
    print(events_elapsed[i])
    print(events_actual_next[i])
    
    print(predicted_events[i])
    
    print(actual_sentence)
    print(predicted_sentence)
    print()
    print()


[['martha', 'supper', 'is', 'na', 'she', 'stew', 'is', 'na', 'she', 'nap', 'decides', 'na', 'she', 'na', 'wakes', 'house']]
['martha', 'na', 'had', 'na']
['she', 'na', 'was', 'na']
It was still a lot of fun.

she was swinging outside .


[['marc', 'na', 'moved', 'friends', 'he', 'them', 'missed', 'na', 'he', 'them', 'invited', 'vacation', 'they', 'na', 'went', 'francisco']]
['marc', 'na', 'was', 'happy']
['he', 'na', 'is', 'happy']
Then I continued the workout.

he is very happy today .


[['jim', 'na', 'wanted', 'na', 'he', 'chance', 'would', 'na', 'jim', 'mixtapes', 'made', 'na', 'he', 'na', 'got', 'famous']]
['it', 'him', 'stopped', 'na']
['he', 'na', 'knew', 'able']
He now sets his cruise control so he won't speed again.

when he got too , his dad realized it was worth just too .




In [26]:
from nltk.translate.bleu_score import sentence_bleu


In [27]:
prediction_model_1 = predicted_events

In [28]:
total_bleu_score = 0

for i in range(len(actual_sentences)):
    print(i)
    actual_sentence = actual_sentences[i]
    predicted_sentence = result['text'][i].numpy().decode()
    predicted_sentence = predicted_sentence.split()
    actual_sentence = [actual_sentence.split()]
    print(actual_sentence)
    print(predicted_sentence)
    score = sentence_bleu(actual_sentence, predicted_sentence, weights=(1,0,0,0))
    total_bleu_score += score
    # print(score)

0
[['It', 'was', 'still', 'a', 'lot', 'of', 'fun.']]
['she', 'was', 'swinging', 'outside', '.']
1
[['Then', 'I', 'continued', 'the', 'workout.']]
['he', 'is', 'very', 'happy', 'today', '.']
2
[['He', 'now', 'sets', 'his', 'cruise', 'control', 'so', 'he', "won't", 'speed', 'again.']]
['when', 'he', 'got', 'too', ',', 'his', 'dad', 'realized', 'it', 'was', 'worth', 'just', 'too', '.']
3
[['He', 'told', 'the', 'boss', 'he', 'already', 'had', 'a', 'lunch', 'plans.']]
['tom', 'knew', 'he', 'wasnt', 'able', 'to', 'continue', 'the', 'night', 'out', '.']
4
[['Jake', 'lived', 'a', 'rich', 'life', 'as', 'a', 'Viking', 'warrior.']]
['the', 'delivery', 'begrudgingly', ',', 'it', 'seemed', 'great', 'to', 'her', '.']
5
[['And', 'they', 'swore', "they'd", 'never', 'get', 'out', 'of', 'touch', 'again.']]
['he', 'is', 'extremely', 'happy', 'with', 'this', 'new', 'work', '!']
6
[['It', 'wonderfully', 'made', 'a', 'nice', 'flame!']]
['she', 'is', 'happy', 'to', 'go', 'get', 'the', 'open', 'oven', 'for', 

/Users/aabhas/.local/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/aabhas/.local/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/aabhas/.local/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF

[['Jake', 'made', 'food', 'for', 'everyone.']]
['he', 'left', 'the', 'place', 'behind', 'her', 'and', 'it', 'smelled', 'fun', '.']
640
[['But', 'it', 'was', 'too', 'late', 'to', 'reverse', 'it.']]
['he', 'said', 'no', '.']
641
[['By', 'the', 'end', 'of', 'the', 'weekend,', 'we', 'were', 'exhausted', 'and', 'happy.']]
['they', 'werent', 'sisters', '.']
642
[['The', 'entire', 'countertop', 'and', 'cabinets', 'were', 'destroyed.']]
['he', 'wound', 'up', 'getting', 'more', 'than', 'almost', 'used', 'to', 'be', 'later', '.']
643
[['The', 'Church', 'thought', 'she', 'was', 'doing', 'so', 'well', 'she', 'was', 'invited.']]
['they', 'were', 'all', 'brain', 'teasers', '.']
644
[['He', 'had', 'thankfully', 'found', "Anna's", 'dog!']]
['she', 'was', 'very', 'disappointed', '.']
645
[['Ricky', 'and', 'his', 'family', 'had', 'a', 'wonderful', 'Sunday.']]
['they', 'were', 'all', 'brain', 'teasers', '.']
646
[['Colin', 'lost', 'his', 'ticket', 'and', 'had', 'to', 'pay', 'a', 'large', 'fine.']]
['i', 

2022-03-13 16:21:09.604185: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 1054 of dimension 0 out of bounds.


InvalidArgumentError: slice index 1054 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [29]:
print(total_bleu_score)

42.28885049878999


In [30]:
average_score_combined_model  = total_bleu_score / len(actual_sentences)
print(average_score_combined_model)

0.028192566999193328


In [31]:
def get_features_and_labels(input_sequences):
    X = input_sequences[:,:-1]
    Y = tf.keras.utils.to_categorical(input_sequences[:,-1], num_classes=len(ids_from_words.get_vocabulary()))
    return X, Y

In [43]:
import os
with open('saved_sequences/test_events.npy', 'rb') as f:
    fsz = os.fstat(f.fileno()).st_size
    input_test_sequences = np.load(f)
    while f.tell() < fsz:
        input_test_sequences = np.vstack((input_sequences, np.load(f)))
        

NameError: name 'get_dataset' is not defined

In [33]:
print(input_test_sequences)

[[   0    0    0 ...    0 5873 9417]
 [   0    0    0 ... 5873 9417 4897]
 [   0    0    0 ... 9417 4897 6307]
 ...
 [   0    0 3294 ... 9399 3294 6307]
 [   0 3294 6307 ... 3294 6307 3792]
 [3294 6307  579 ... 6307 3792 6307]]


In [34]:
class Event2Event(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstmunits = tf.keras.layers.LSTM(rnn_units,
                                       return_sequences=False,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.lstmunits.get_initial_state(x)
        
        x, memory_state, carry_state = self.lstmunits(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, memory_state, carry_state
        else:
            return x

In [35]:
# Length of the vocabulary in chars
vocab_size = len(ids_from_words.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [36]:
model = Event2Event(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_words.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [44]:
loss_fn = tf.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [45]:
model.compile(optimizer=optimizer, loss=loss_fn)

In [46]:
# Batch size
BATCH_SIZE = 256

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1024

def get_dataset(X, Y):
    dataset =  tf.data.Dataset.from_tensor_slices((X,Y))
    dataset = (dataset
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE, drop_remainder=False)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return dataset
x_test, y_test = get_features_and_labels(input_test_sequences)

test_dataset = get_dataset(x_test, y_test)

In [47]:
model.load_weights('trained_events_model_second_batch_validation_1/cp.ckpt')

In [48]:
# model.evaluate(input_test_sequences)

In [81]:
loss = model.evaluate(x_test, y_test, batch_size=128)

157/157 [==============================] - 3s 18ms/step - loss: 7.1539


In [82]:
print(loss)

7.153858184814453


In [83]:
combined_perplexity_score = tf.exp(loss)

In [84]:
print(combined_perplexity_score)

tf.Tensor(1279.0306, shape=(), dtype=float32)


In [69]:
average_score_combined_model

0.028192566999193328

In [48]:
predicted_events

[['she', 'it', 'decided', 'na'],
 ['i', 'na', 'know', 'na'],
 ['na', 'na', 'is', 'able'],
 ['he', 'na', 'was', 'able'],
 ['they', 'na', 'were', 'na'],
 ['they', 'na', 'were', 'na'],
 ['it', 'na', 'is', 'able'],
 ['i', 'na', "'ll", 'na'],
 ['he', 'na', 'knew', 'na'],
 ['she', 'it', 'was', 'able'],
 ['mary', 'na', 'said', 'na'],
 ['barry', 'na', 'falls', 'him'],
 ['he', 'na', 'was', 'able'],
 ['fred', 'him', 'asks', 'him'],
 ['she', 'it', 'was', 'able'],
 ['kid', 'na', 'loses', 'na'],
 ['it', 'na', 'was', 'able'],
 ['i', 'na', "'ve", 'na'],
 ['it', 'na', 'made', 'na'],
 ['i', 'na', 'ended', 'na'],
 ['they', 'na', 'had', 'na'],
 ['she', 'na', 'was', 'excited'],
 ['he', 'na', 'decided', 'scared'],
 ['jim', 'him', 'told', 'able'],
 ['na', 'na', 'burnt', 'na'],
 ['she', 'na', 'felt', 'embarrassed'],
 ['she', 'na', 'is', 'embarrassed'],
 ['i', 'na', 'could', 'na'],
 ['na', 'na', 'awed', 'size'],
 ['they', 'na', 'were', 'able'],
 ['she', 'job', 'found', 'her'],
 ['he', 'na', 'is', 'happy'],
 [

In [49]:
actual_sentences

['It was still a lot of fun.\n',
 'Then I continued the workout.\n',
 "He now sets his cruise control so he won't speed again.\n",
 'He told the boss he already had a lunch plans.\n',
 'Jake lived a rich life as a Viking warrior.\n',
 "And they swore they'd never get out of touch again.\n",
 'It wonderfully made a nice flame!\n',
 'When i first met him he was so sweet and gentleman.\n',
 'Tim had to have it removed at a hospital.\n',
 'Suddenly she saw the perfect affordable toy.\n',
 "When Mary couldn't provide a doctor's note, she failed the exam.\n",
 "Suddenly Barry wasn't so hungry anymore.\n",
 "He didn't win anything on those, though.\n",
 'James is happy that he can be of service to others each and every day.\n',
 "Just as she was about to order, she realized she'd lost her purse too.\n",
 "Ken's kid was better about it after.\n",
 'But as he drove his rc car back home, it got run over.\n',
 'I ended up cleaning the entire house from the energy.\n',
 'We all had a blast throwin

In [70]:
total_bleu_score_model_1 = 0

for i in range(len(actual_sentences)):
    print(i)
    actual_sentence = actual_sentences[i]
    predicted_sentence = predicted_events[i]
    actual_sentence = [actual_sentence.split()]
    print(actual_sentence)
    print(predicted_sentence)
    score = sentence_bleu(actual_sentence, predicted_sentence, weights=(1,0,0,0))
    total_bleu_score_model_1 += score

0
[['It', 'was', 'still', 'a', 'lot', 'of', 'fun.']]
['she', 'na', 'was', 'na']
1
[['Then', 'I', 'continued', 'the', 'workout.']]
['he', 'na', 'is', 'happy']
2
[['He', 'now', 'sets', 'his', 'cruise', 'control', 'so', 'he', "won't", 'speed', 'again.']]
['he', 'na', 'knew', 'able']
3
[['He', 'told', 'the', 'boss', 'he', 'already', 'had', 'a', 'lunch', 'plans.']]
['tom', 'na', 'knew', 'able']
4
[['Jake', 'lived', 'a', 'rich', 'life', 'as', 'a', 'Viking', 'warrior.']]
['it', 'na', 'felt', 'great']
5
[['And', 'they', 'swore', "they'd", 'never', 'get', 'out', 'of', 'touch', 'again.']]
['na', 'na', 'is', 'happy']
6
[['It', 'wonderfully', 'made', 'a', 'nice', 'flame!']]
['she', 'na', 'is', 'happy']
7
[['When', 'i', 'first', 'met', 'him', 'he', 'was', 'so', 'sweet', 'and', 'gentleman.']]
['he', 'na', 'is', 'na']
8
[['Tim', 'had', 'to', 'have', 'it', 'removed', 'at', 'a', 'hospital.']]
['she', 'na', 'went', 'na']
9
[['Suddenly', 'she', 'saw', 'the', 'perfect', 'affordable', 'toy.']]
['i', 'na', 

IndexError: list index out of range

In [71]:
total_bleu_score_model_1

8.167815105365596

In [72]:
average_total_bleu_score_model_1 = total_bleu_score_model_1/ len(actual_sentences)

In [73]:
average_total_bleu_score

0.005445210070243731

In [74]:
model_2_actual_input = tf.constant(actual_sentences)

In [75]:
model_2_actual_input

<tf.Tensor: shape=(1500,), dtype=string, numpy=
array([b'It was still a lot of fun.\n',
       b'Then I continued the workout.\n',
       b"He now sets his cruise control so he won't speed again.\n", ...,
       b'Next weekend she will see if she can find a matching mirror.\n',
       b'The workers were annoyed with the ordeal.\n',
       b'I saved a lot of money by being thrifty.\n'], dtype=object)>

In [76]:
actual_result = text_translator.tf_translate(
    input_text = model_2_actual_input)

In [85]:
total_bleu_score_model_2= 0

for i in range(len(actual_sentences)):
    # print(i)
    actual_sentence = actual_sentences[i]
    predicted_sentence = actual_result['text'][i].numpy().decode()
    predicted_sentence = predicted_sentence.split()
    actual_sentence = [actual_sentence.split()]
    print(actual_sentence)
    print(predicted_sentence)
    score = sentence_bleu(actual_sentence, predicted_sentence, weights=(1,0,0,0))
    total_bleu_score_model_2 += score

[['It', 'was', 'still', 'a', 'lot', 'of', 'fun.']]
['it', 'was', 'so', 'in', 'class', 'as', 'i', 'could', 'school', '.']
[['Then', 'I', 'continued', 'the', 'workout.']]
['when', 'the', 'boss', 'started', ',', 'his', 'neighbor', 'learned', 'that', 'it', 'was', 'a', 'bad', 'name', 'on', 'today', '.']
[['He', 'now', 'sets', 'his', 'cruise', 'control', 'so', 'he', "won't", 'speed', 'again.']]
['he', 'now', 'beatrice', 'sharing', 'his', 'compass', 'and', 'since', 'it', 'master', 'was', 'all', 'another', '.']
[['He', 'told', 'the', 'boss', 'he', 'already', 'had', 'a', 'lunch', 'plans.']]
['when', 'he', 'got', 'home', ',', 'and', 'i', 'was', 'crying', 'from', 'his', 'mom', 'to', 'answer', 'the', 'keys', '.']
[['Jake', 'lived', 'a', 'rich', 'life', 'as', 'a', 'Viking', 'warrior.']]
['jake', 'never', 'got', 'paintball', 'due', 'to', 'love', '.']
[['And', 'they', 'swore', "they'd", 'never', 'get', 'out', 'of', 'touch', 'again.']]
['one', 'reviewer', 'always', 'proved', 'everyone', 'to', 'do', 'a

In [78]:
model_2_bleu_score = total_bleu_score_model_2 / len(actual_sentences)
model_2_bleu_score

0.10062260125473423